# Strategyパターン
*  Strategy（ストラテジー）はアルゴリズム（処理の手順）
*  Strategyパターンは，複数の様々なストラテジーを定義し，それらを動的に交換可能とすることができる（実行時に選択できる）
*  Strategyパターンは，アルゴリズムの詳細をカプセル化し，利用者から分離させることができるので，アルゴリズムの交換や拡張を容易にできる
*  その結果，コードの再利用性が向上する

## Strategyパターンの構成要素
*   **Strategy**: アルゴリズム共通のインターフェース．具体的なアルゴリズムは，すべてこのインターフェースを継承する
*   **ConcreteStrategy**: ストラテジーを実装し，具体的な複数のストラテジーを実装する
*   **Context**: 
>*   具体的なストラテジーを選択・使用するクラスで，動的にアルゴリズムを設定または交換できるようにする
>*   このとき，Contextはアルゴリズムの具体的な実装を知らず，Strategyインターフェースを通じてアルゴリズムにアクセスする

<img src="./fig/design_patterns_strategy.jpg" width="550">
  
---
**基本的な書式:**
```Python
from abc import ABC, abstractmethod

# Strategy
class Strategy(ABC):
    
    @abstractmethod
    def execute(self):
        pass

# ConcreteStrategyA
class ConcreteStrategyA(Strategy):
    
    def execute(self):
        ConcreteStrategyAのストラテジー

# ConcreteStrategyB
class ConcreteStrategyB(Strategy):
    
    def execute(self):
        ConcreteStrategyBのストラテジー

# Context
class Context:
    
    def __init__(self, strategy: Strategy):
        self.__strategy = strategy

    
    def perform(self):
        self.__strategy.execute() # 具体的なストラテジーの実行
```
---


## Strategyパターンの例 その1
*  以下のコードは，見出しと箇条書きを様々な表示形式（フォーマット）で出力するシステムを簡易的に実現している
*  このシステムでは，タイトルと各項目のフォーマット（例えば，HTML形式やマークダウン形式など）を動的に変更できる
*  この例では，`Formatter`がStrategyとして機能し，`HTMLFormatter`と`MarkdownFormatter`がこのインターフェースを実装している
*  `Itemizer`クラスはContextとして機能し，見出しと箇条書きを表示するためにストラテジーを使用する
*  利用者側（クライアント）のコードでは，`Itemizer`クラスのインスタンスを異なるストラテジー（HTML形式やマークダウン形式）で作成し，見出しと箇条書きを表示している

> <img src="./fig/design_patterns_strategy_example.jpg" width="700">

In [ ]:
from abc import ABC, abstractmethod

# Strategy
class Formatter(ABC):
    @abstractmethod
    def output_contents(self, title: str, items: list[str]) -> None:
        pass

# ConcreteStrategy
class HTMLFormatter(Formatter):
    def output_contents(self, title: str, items: list[str]) -> None:
        print(f'<body>\n<h1>{title}</h1>\n<ul>')
        for item in items:
            print(f'<li>{item}</li>')
        print('</ul>\n</body>')

# ConcreteStrategy
class MarkdownFormatter(Formatter):
    def output_contents(self, title: str, items: list[str]) -> None:
        print(f'# {title}')
        for item in items:
            print(f'*   {item}')

# Context
class Itemizer:
    def __init__(self, title: str, items: list[str], formatter: Formatter) -> None:
        self.__title = title
        self.__items = items
        self.__strategy = formatter

    def perfom(self) -> None:
        self.__strategy.output_contents(self.__title, self.__items)

# クライアント
title: str = 'News & Events'
contents: list[str] = ['January 15, 2024: ...', 'March 25, 2024: ...']
item_html: Itemizer = Itemizer(title, contents, HTMLFormatter()) #アルゴリズム（ストラテジー）の選択
item_html.perfom()
print('-' * 50) # 区切り線
item_markdown: Itemizer = Itemizer(title, contents, MarkdownFormatter()) #アルゴリズム（ストラテジー）の選択
item_markdown.perfom()

## Strategyパターンの例 その2
* ここでは，OCP（開放・閉鎖の原則）に違反しているコードを，Strategyパターンを使ってOCP遵守のコードに改善する例を考える

### OCPに違反している例
* 以下のコードは，通常価格（`base_price`）から割引種別に応じた価格を計算する `calc_price`関数を定義している
* 割引種別は，none（割引なし），seasonal（季節割）, vip（VIP割） の3種
* `calc_price`関数は，`if`文の条件分岐で割引種別を判定し，対応する価格の計算処理を行っている
* しかし，例えば，coupon（クーポン割）などの新しい割引を追加するたびに，この関数を編集する必要があり，既存コードの修正に開いている
* したがって，このコードは OCP に違反している

In [ ]:
def calc_price(base_price: int, discount: str) -> int:
    if discount == 'none':
        return base_price
    elif discount == 'seasonal':
        return int(base_price * 0.9)   # 季節割引：10%OFF
    elif discount == 'vip':
        return int(base_price * 0.8)   # VIP割引：20%OFF
    else:
        raise ValueError('未知の割引種別です')
    
# クライアント
print(calc_price(1000, 'none'))
print(calc_price(1000, 'seasonal'))
print(calc_price(1000, 'vip'))

### OCPを遵守したコードに改善
* ここで，Strategyパターンを使ってOCPを遵守したコードに改善する
* 改善したコードは以下のとおりで，クラス図は下図のように描ける

> <img src="./fig/Strategy_example2.jpg" width="750">

* 3つの割引種別は，ConcreteStrategy の役割を担う 3つのクラス `NoDiscount`（割引なし）, `SeasonalDiscount`（季節割）, `VipDiscount`（VIP割） に対応する
* これら割引種別を抽象化したものが Strategy の役割を担うインターフェース `DiscountStrategy`
* `DiscountStrategy` の中で，価格を計算する抽象メソッド `apply` を定義
* 実装は `NoDiscount`, `SeasonalDiscount`, `VipDiscount` で定義
* Context の役割を担う `PriceCalculator`クラスが戦略（割引種別）を選択・保持する
* 戦略は，コンストラクタ（`__init__`）と `set_strategy`メソッドで選択し，インスタンス属性 `__strategy` として保持する
* 価格は，インスタンス属性 `__strategy` に応じて `calc`メソッドの中で計算される
* 割引種別の変更は，クライアント側で`set_strategy`メソッドを呼び出すだけでよい

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional

# Strategy
class DiscountStrategy(ABC):
    @abstractmethod
    def apply(self, base_price: int) -> int:
        pass

# ConcreteStrategy
class NoDiscount(DiscountStrategy):
    def apply(self, base_price: int) -> int:
        return base_price

# ConcreteStrategy
class SeasonalDiscount(DiscountStrategy):
    def apply(self, base_price: int) -> int:
        return int(base_price * 0.9)

# ConcreteStrategy
class VipDiscount(DiscountStrategy):
    def apply(self, base_price: int) -> int:
        return int(base_price * 0.8)

# Context
class PriceCalculator:
    def __init__(self, strategy: DiscountStrategy) -> None:
        self.__strategy: DiscountStrategy = strategy

    def set_strategy(self, strategy: DiscountStrategy) -> None:
        self.__strategy = strategy

    def calc(self, base_price: int) -> int:
        return self.__strategy.apply(base_price)

# クライアント
calc = PriceCalculator(NoDiscount())
print(calc.calc(1000))

calc.set_strategy(SeasonalDiscount())
print(calc.calc(1000))

calc.set_strategy(VipDiscount())
print(calc.calc(1000))

### 新しい戦略（割引種別）の追加
* 上のコードに新しい割引種別として，`CouponDiscount`（クーポン割）を追加する
* クーポン割の価格は，通常価格から割引金額を引いた金額とする（マイナスになった場合は 0 とする）
* 割引金額は任意に設定できるものとし，`CouponDiscount`のインスタンス属性`__amount_off`に格納しておく
* 以下のコードが，クーポン割を追加したコードとなるが，他のクラスには一切影響していないこと（OCP遵守）が確認できる

> <img src="./fig/Strategy_example3.jpg" width="800">

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional

# Strategy
class DiscountStrategy(ABC):
    @abstractmethod
    def apply(self, base_price: int) -> int:
        pass

# ConcreteStrategy
class NoDiscount(DiscountStrategy):
    def apply(self, base_price: int) -> int:
        return base_price

# ConcreteStrategy
class SeasonalDiscount(DiscountStrategy):
    def apply(self, base_price: int) -> int:
        return int(base_price * 0.9)

# ConcreteStrategy
class VipDiscount(DiscountStrategy):
    def apply(self, base_price: int) -> int:
        return int(base_price * 0.8)

# ConcreteStrategy（新規追加）
class CouponDiscount(DiscountStrategy):
    def __init__(self, amount_off: int) -> None:
        self.__amount_off: int = amount_off

    def apply(self, base_price: int) -> int:
        return max(0, base_price - self.__amount_off)

# Context
class PriceCalculator:
    def __init__(self, strategy: DiscountStrategy) -> None:
        self.__strategy: DiscountStrategy = strategy

    def set_strategy(self, strategy: DiscountStrategy) -> None:
        self.__strategy = strategy

    def calc(self, base_price: int) -> int:
        return self.__strategy.apply(base_price)

# クライアント
calc = PriceCalculator(NoDiscount())
print(calc.calc(1000))

calc.set_strategy(SeasonalDiscount())
print(calc.calc(1000))

calc.set_strategy(VipDiscount())
print(calc.calc(1000))

calc.set_strategy(CouponDiscount(150)) # クーポン割（150円引き）を選択
print(calc.calc(1000))

## Strategyパターンのメリット・デメリット

| 観点       | メリット                                             | デメリット                             |
| ------------ | ------------------------------------------------ | --------------------------------- |
| **拡張のしやすさ**  | 新しいアルゴリズムをクラス追加で実現でき，既存コードを修正せずに拡張できる（OCP遵守） | クラスの数が増えてコードが散らかりやすい              |
| **責任分離**     | 各アルゴリズムが独立したクラスになり，単一責任の原則（SRP）を満たしやすい         | Strategyが細分化しすぎると全体像を把握しにくくなる     |
| **切り替えの柔軟性** | 実行時にアルゴリズムを差し替え可能（条件分岐より柔軟）                      | 利用側（Context）が常にStrategyを意識する必要がある |
| **テストのしやすさ** | 各Strategyを個別にテストできる                        | 共通の挙動確認が重複しやすい                    |
| **再利用性**     | Strategyを他のコンテキストでも再利用できる                        | Strategy間でコードの重複が発生する可能性がある       |

**ポイント**:
* Strategyは「**条件分岐をなくして多態性で置き換える**」のが基本的な狙い
* 特に **OCP** と **SRP** に強い ⇒ 保守性・テスト性が向上する
* ただし小規模コードでは「クラスが増えて逆に煩雑」と見えることがあるので，使う場面の見極めが重要

# Decoratorパターン
*  OCP（開放・閉鎖の原則）と密接に関連するデザインパターンの一つ
*  オブジェクトの機能を実行時に動的に拡張または変更するために使用される
*  Decoratorパターンは，既存のオブジェクトの構造を変更せずに，オブジェクトに新しい機能を柔軟に追加するための効果的な方法である
*  このパターンは，OCPに従い，オブジェクトの機能拡張を可能する
*  これまでに使ってきた，Pythonの組み込みデコレータ「`@デコレータ`」もデコレータパターンに基づいて実装されている

##  Decoratorパターンの構成要素
*   Compornent:
>*  基本的な機能を提供するインターフェースまたは抽象クラス
>*  これに新しい機能（`operation`）が追加される
>*  `operation`は抽象メソッド
*   ConcreteCompornent:
>*  Compornentを継承したクラス
>*  Compornentで定義した機能である`operation`メソッドを実装して基本的な機能を提供する
*   Decorator:
>*  Compornentを継承したクラスで，構成要素としてComponentを持つ
>*  構成要素であるComponentに新しい機能を追加したクラスの共通のスーパークラスとして機能する
*   ConcreteDecorator:
>*  Decoratorを継承したクラス
>*  Decoratorの処理（`operation`メソッド）を具体的に実装する
>*  `operation`メソッドは，ConcreteComponentの機能（`operation`メソッド），またはそれをデコレートした機能となる
>*  つまり，デコレートした機能を，さらにデコレートすることもできる

<img src="./fig/design_patterns_decorator.png" width="400">

## Decoratorパターンの例
* ここでは，OCP（開放・閉鎖の原則）に違反しているコードを，Decoratorパターンを使ってOCP遵守のコードに改善する例を考える

### OCPに違反している例
* 以下のコードは，通常価格（`base_price`）からオプションに応じた価格を計算する `calc_price`関数（前述した例で登場した `calc_price`関数とは異なる関数）を定義している
* オプションには，「tax（税）」「seasonal_discount（季節割）」「vip_discount（VIP割）」「handling_fee（手数料）」があり，任意の組み合わせで価格が計算できる（適用順序も任意）
* ここで，税⇒10%，季節割⇒10%引き，VIP割⇒20%引き，手数料⇒150円追加とした
* 任意の組み合わせによる価格計算は，`calc_price`関数を複数回呼び出すことで実現できる
* `calc_price`関数は，`if`文の条件分岐でオプションを判定し，対応する価格の計算処理を行っている
* しかし，例えば，「coupon（クーポン割）」などの新しいオプションを追加するたびに，この関数を編集する必要があり，既存コードの修正に開いている
* したがって，このコードは OCP に違反している

In [ ]:
def calc_price(base_price: int, opt: str) -> int:
    price: int = base_price
    if opt == 'tax':
        price = int(price * 1.1)  # 10%税込
    elif opt == 'seasonal_discount':
        price = int(price * 0.9)  # 10%引き
    elif opt == 'vip_discount':
        price = int(price * 0.8)  # 20%引き
    elif opt == 'handling_fee':
        price = price + 150        # 取扱手数料
    else:
        raise ValueError(f'未知のオプション: {opt}')
    return price

# クライアント
# 例1）税のみ
print(f'税のみ: {calc_price(1000, 'tax')}') # 1100

# 例2）税⇒季節割
print(f'税⇒季節割: {calc_price(calc_price(1000, 'tax'), 'seasonal_discount')}') # 1000×1.1×0.9=990

# 例3）税⇒VIP割⇒手数料
print(f'税⇒VIP割⇒手数料: {calc_price(calc_price(calc_price(1000, 'tax'), 'vip_discount'), 'handling_fee')}') # 1000×1.1×0.8+150=1030

### OCPを遵守したコードに改善
* ここで，Decoratorパターンを使ってOCPを遵守したコードに改善する
* 改善したコードは以下のとおりで，クラス図は下図のように描ける

> <img src="./fig/Decorator_example2.jpg" width="650">

*  通常価格に対応するコンポーネントと，その価格にオプションに応じた価格計算を行うデコレータを実装している
*  Compornent:
>*  抽象クラス（インターフェース）`PriceCalculator`
>*  `calc`メソッドを抽象メソッドとして定義
>*  `PriceCalculator`が全てのConcreteCompornentとDecoratorの共通インターフェースとなる
*  ConcreteCompornent:
>*  `PriceCalculator`クラスを継承した`BasePrice`クラスで，デコレータを適用する基礎となるクラス
>*  通常価格をインスタンス属性`__base_price`として持つ
>*  `calc`メソッドの実装を定義する
>*  `calc`メソッドは，`__base_price`の内容を返すシンプルな機能を実現する
*   Decorator:
>*  `PriceCalculator`クラスを継承した抽象クラス`PriceDecorator`
>*  Compornentである`PriceCalculator`クラスを継承したクラスのインスタンス（この例では`BasePrice`や`Tax10Percent`, `SeasonalDiscount10`などのインスタンス）をラップし，追加機能を提供するためのクラス
>*  ラップしたクラスは，インスタンス属性`__inner`として保持する
>*  このクラスは，抽象クラスとして定義し，具体的な追加方法（実装）は，サブクラスであるConcreteDecorator（この例では`Tax10Percent`や`SeasonalDiscount10`など）で定義する
*   ConcreteDecorator:
>*  Decoratorである`PriceDecorator`クラスを継承したクラス ⇒ `Tax10Percent`, `SeasonalDiscount10`, `VipDiscount20`, `HandlingFee150`
>*  Decoratorの処理（`calc`メソッド）を具体的に実装する
>*  `calc`メソッドは，インスタンス属性`__inner`として保持しているクラスの機能（`calc`メソッド）をデコレートしたメソッドとなる


In [ ]:
from abc import ABC, abstractmethod
from typing import Optional

# Component
class PriceCalculator(ABC):
    @abstractmethod
    def calc(self) -> int:
        pass

# ConcreteComponent: 通常価格
class BasePrice(PriceCalculator):
    def __init__(self, base_price: int) -> None:
        self.__base_price: int = base_price

    def calc(self) -> int:
        return self.__base_price

# Decorator
class PriceDecorator(PriceCalculator):
    def __init__(self, inner: PriceCalculator) -> None:
        self._inner: PriceCalculator = inner

    @abstractmethod
    def calc(self) -> int:
        pass

# ConcreteDecorator: 税込み価格を計算
class Tax10Percent(PriceDecorator):
    def calc(self) -> int:
        price: int = self._inner.calc()
        return int(price * 1.10)

# ConcreteDecorator: 季節割の適用価格を計算
class SeasonalDiscount10(PriceDecorator):
    def calc(self) -> int:
        price: int = self._inner.calc()
        return int(price * 0.90)

# ConcreteDecorator: VIP割の適用価格を計算
class VipDiscount20(PriceDecorator):
    def calc(self) -> int:
        price: int = self._inner.calc()
        return int(price * 0.80)

# ConcreteDecorator: 手数料込みの価格を計算
class HandlingFee150(PriceDecorator):
    def calc(self) -> int:
        price: int = self._inner.calc()
        return price + 150

# クライアント
base: PriceCalculator = BasePrice(1000) # 型ヒントはサブクラスを含むスーパークラスにする

# 例1）税のみ
case1: PriceCalculator = Tax10Percent(base)
print(f'税のみ: {case1.calc()}') # 1100

# 例2）税⇒季節割
case2: PriceCalculator = SeasonalDiscount10(case1)
print(f'税⇒季節割: {case2.calc()}') # 1000×1.1×0.9=990

# 例3）税⇒VIP割⇒手数料
case3: PriceCalculator = HandlingFee150(VipDiscount20(case1))
print(f'税⇒VIP割⇒手数料: {case3.calc()}') # 1000×1.1×0.8+150=1030

### 新しいオプションの追加
* 上のコードに新しいオプションとして，`CouponDiscount`（クーポン割）を追加する
* クーポン割の価格は，通常価格から割引金額を引いた金額とする（マイナスになった場合は 0 とする）
* 割引金額は任意に設定できるものとし，`CouponDiscount`のインスタンス属性`__amount_off`に格納しておく
* 以下のコードが，クーポン割を追加したコードとなるが，他のクラスには一切影響していないこと（OCP遵守）が確認できる

> <img src="./fig/Decorator_example3.jpg" width="800">

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional

# Component
class PriceCalculator(ABC):
    @abstractmethod
    def calc(self) -> int:
        pass

# ConcreteComponent: 通常価格
class BasePrice(PriceCalculator):
    def __init__(self, base_price: int) -> None:
        self.__base_price: int = base_price

    def calc(self) -> int:
        return self.__base_price

# Decorator
class PriceDecorator(PriceCalculator):
    def __init__(self, inner: PriceCalculator) -> None:
        self._inner: PriceCalculator = inner

    @abstractmethod
    def calc(self) -> int:
        pass

# ConcreteDecorator: 税込み価格を計算
class Tax10Percent(PriceDecorator):
    def calc(self) -> int:
        price: int = self._inner.calc()
        return int(price * 1.10)

# ConcreteDecorator: 季節割の適用価格を計算
class SeasonalDiscount10(PriceDecorator):
    def calc(self) -> int:
        price: int = self._inner.calc()
        return int(price * 0.90)

# ConcreteDecorator: VIP割の適用価格を計算
class VipDiscount20(PriceDecorator):
    def calc(self) -> int:
        price: int = self._inner.calc()
        return int(price * 0.80)

# ConcreteDecorator: 手数料込みの価格を計算
class HandlingFee150(PriceDecorator):
    def calc(self) -> int:
        price: int = self._inner.calc()
        return price + 150

# ConcreteDecorator: クーポン割りの適法価格を計算（新規追加）
class CouponDiscount(PriceDecorator):
    def __init__(self, inner: PriceCalculator, amount_off: int) -> None:
        super().__init__(inner)
        self.__amount_off: int = amount_off

    def calc(self) -> int:
        price: int = self._inner.calc()
        return max(0, price - self.__amount_off)
    

# クライアント
base: PriceCalculator = BasePrice(1000) # 型ヒントはサブクラスを含むスーパークラスにする

# 例1）税のみ
case1: PriceCalculator = Tax10Percent(base)
print(f'税のみ: {case1.calc()}') # 1100

# 例2）税⇒季節割
case2: PriceCalculator = SeasonalDiscount10(case1)
print(f'税⇒季節割: {case2.calc()}') # 1000×1.1×0.9=990

# 例3）税⇒VIP割⇒手数料
case3: PriceCalculator = HandlingFee150(VipDiscount20(case1))
print(f'税⇒VIP割⇒手数料: {case3.calc()}') # 1000×1.1×0.8+150=1030

# 例4）税⇒クーポン割
case4: PriceCalculator = CouponDiscount(case1, 150)
print(f'税⇒クーポン割: {case4.calc()}') # 1000×1.1-150=950

## Decoratorパターンのメリット・デメリット

| 観点           | メリット                                        | デメリット                              |
| ---------------- | ------------------------------------------- | ---------------------------------- |
| **拡張のしやすさ**      | 新しい機能を「クラス追加」で拡張でき．既存コードを修正せずにすむ（OCP遵守） | デコレータの数が増えるとクラスが多くなり複雑に見える         |
| **柔軟な組み合わせ**     | 複数のデコレータを自由に組み合わせられる（積み木のように使える）            | 適用順序によって結果が変わることがあり，設計者が意図を理解しにくい  |
| **責任集中** | 各デコレータが「1つの機能追加」に集中できる（SRP遵守）                      | 細分化しすぎるとクラスが増大し，把握しにくい         |
| **再利用性**         | デコレータを使い回せる                       | 特定のコンポーネントにしか使えないデコレータを作ると再利用性が下がる |
| **動的な機能付与**      | 実行時にオブジェクトへ機能を追加できる（サブクラス化より柔軟）             | 実行時のラップ構造を追いにくくなり，デバッグが難しくなる       |

**ポイント**:
* Decoratorは「継承の代わりに合成で拡張」する仕組み
* OCPとSRPを強化でき，保守性・柔軟性が高まる
* その一方で「クラス数の爆発」「ラップの深さが分かりにくい」が典型的なデメリット


# リスコフの置換原則（LSP: Liskov substitution principle）
*  SOLID原則の一つであるリスコフの置換原則（LSP: Liskov substitution principle）は，1988年，Barbara Liskovによって提唱された原則で，オブジェクト指向プログラミングにおける継承の正しい利用方法を示している ⇒  原著論文「[Data abstraction and hierarchy](https://dl.acm.org/doi/10.1145/62139.62141)」（学会での基調講演）
>*  「スーパークラスのオブジェクトをサブクラスのオブジェクトで置き換えても，プログラムの正当性が保たれるべきである．」
*  より形式的な定義は，LiskovとWingによる1994年の論文「[A behavioral notion of subtyping](https://dl.acm.org/doi/10.1145/197320.197383)」で与えられている
*  LSPは，サブクラスがスーパークラスの代わりとして機能することを保証する
*  例えば，ある関数がスーパークラスのインスタンスを引数として呼び出せるなら，サブクラスのインスタンスを引数としても呼び出せなければならない
*  サブクラスがスーパークラスの振る舞いを不適切に変更する場合，LSPに違反することになる

## リスコフの置換原則を遵守することによる効果
*  コードの再利用性の向上: サブクラスをスーパークラスの代わりに使用できるため，既存のコードを再利用しやすくなる
*  保守性の向上: サブクラスがスーパークラスの期待する振る舞いを維持するため，バグの発生リスクが低減する
*  柔軟性の確保: クラス階層が正しく設計されていると，システム全体の柔軟性が向上し，新しい機能の追加がスムーズになる
*  依存関係の明確化: サブクラスがスーパークラスの契約を守ることで，依存関係が明確になり，システムの理解が容易になる

## 具体例: 鳥の種類

### LSPに違反したコード
*  以下のコードは，下図に対応するクラスを実装している

> <img src="./fig/SOLID_LSP_bird1.jpg" width="300">

*  このコードは，サブクラスがスーパークラスの契約を破っているので，リスコフの置換原則に違反している
*  `Bird`クラスの`fly`メソッドを使用することをクライアントは期待しているが，`Ostrich`クラスではそれが破られている
*  ダチョウ（`Ostrich`）は飛べないにもかかわらず，`fly`メソッドを実装する必要がある ⇒ 呼び出すと例外（NotImplementedError）が発生するようにしている
>*  NotImplementedErrorは，主に抽象メソッドや未実装の機能に関連して使用される例外
>*  まだ実装していないメソッドが呼び出されたときなどに使用される


In [ ]:
from abc import ABC, abstractmethod

class Bird(ABC):
    
    @abstractmethod
    def fly(self) -> None:
        pass

class Sparrow(Bird):
    
    def fly(self) -> None:
        print('スズメが飛びます。')

class Ostrich(Bird):
    
    def fly(self) -> None:
        raise NotImplementedError('ダチョウは飛べません。')

# クライアント
def make_bird_fly(bird: Bird) -> None:
    bird.fly()

sparrow: Sparrow = Sparrow()
make_bird_fly(sparrow)  # スズメが飛びます。

ostrich: Ostrich = Ostrich()
# make_bird_fly(ostrich)  # 例外発生: NotImplementedError

### LSPに従ったコードに修正
*  LSPを遵守するためには，設計を見直し，スーパークラスの契約を維持しつつ，適切なクラス階層を構築する必要がある
*  **クラス階層の分割**: `Bird`クラスをスーパークラスとし，飛ぶ鳥用の`FlyingBird`クラスを新たに作成
*  **スーパークラスの契約の維持**: `Ostrich`クラスは，`Bird`クラスを継承し，`move`メソッドを実装
*  ただし，`fly`メソッドは必要ないため，`FlyingBird`クラスを使用して飛べる鳥と飛べない鳥を分ける
*  **契約違反の回避**: `Ostrich`クラスは，`fly`メソッドを持たない（実装する必要がない）ため，クライアントは飛べない鳥に対して`fly`メソッドを呼び出すことはしない

> <img src="./fig/SOLID_LSP_bird2.jpg" width="300">

In [ ]:
from abc import ABC, abstractmethod

class Bird(ABC):
    
    @abstractmethod
    def move(self) -> None:
        pass

class FlyingBird(Bird):
    
    @abstractmethod
    def fly(self) -> None:
        pass

class Sparrow(FlyingBird):
    
    def move(self) -> None:
        print('スズメが歩きます。')
    
    def fly(self) -> None:
        print('スズメが飛びます。')

class Ostrich(Bird):
    
    def move(self) -> None:
        print('ダチョウが走ります。')

# クライアント側
def make_flying_bird_fly(bird: FlyingBird) -> None:
    bird.fly()

sparrow = Sparrow()
make_flying_bird_fly(sparrow)  # 出力: スズメが飛びます。

ostrich = Ostrich()
# make_flying_bird_fly(ostrich)  # エラー: OstrichはFlyingBirdのサブクラスではないのでflyメソッドが使えない


# 実習
以下の「修正前コード」は，「問題点」で指摘しているようにリスコフの置換原則（LSP）に違反している．このコードを「コード修正要件・補足説明」に従って修正し，LSP遵守のコードに修正しなさい．ただし，コードには型ヒントをつけ，既に入力されているコードは変更・削除しないこと．

---
**修正前コード：**
```Python
from abc import ABC, abstractmethod

class Payment(ABC):
    @abstractmethod
    def pay(self, amount: int) -> None:
        pass

    @abstractmethod
    def refund(self, amount: int) -> None:
        pass

class CreditCardPayment(Payment):
    def pay(self, amount: int) -> None:
        print(f'クレカ決済: {amount}円')
    def refund(self, amount: int) -> None:
        print(f'クレカ返金: {amount}円')

class CashPayment(Payment):
    def pay(self, amount: int) -> None:
        print(f'現金受領: {amount}円')
    def refund(self, amount: int) -> None:
        raise NotImplementedError('現金はオンライン返金に対応していません')
```
---
**問題点:**
* このコードでは，「支払い」に対する共通のインターフェース`Payment`で，2つの抽象メソッド`pay`（支払い処理に対応）と`refund`（返金処理に対応）を定義している ⇒ これがLSP違反の原因となる
* 理由: `pay`と`refund`を実装するクラスとして，`CreditCardPayment`（クレジットカード利用）と`CashPayment`（現金払い）を定義しているが，現金払いの場合は返金できないことにしているので，`CashPayment`の`refund`が呼び出されると，例外`NotImplementedError`を発生させている（前述した鳥の種類の例の`Ostrich`クラスと同様）
* つまり，スーパークラスが約束した契約をサブクラスが守れていない
---
**コード修正要件・補足説明:**
*  以下のクラス図と対応させる
*  `pay`メソッドと`refund`メソッドの実装は修正前コードとすべて同じ
---
**クラス図:**

<img src='./fig/SOLID_LSP_exercise.jpg' width='350'>

---
**期待される実行結果:**
```
クレカ決済: 1000円
クレカ返金: 1000円
現金受領: 5000円
```

In [ ]:
# ここにコードを記述（必要に応じて改行する）

# --------------------------
# 動作確認（クライアント側）
# --------------------------
trans1: Payment = CreditCardPayment()
trans1.pay(1000)
trans1.refund(1000)

trans2: Payment = CashPayment()
trans2.pay(5000)

# 参考資料
*  Robert C.Martin (著), 角征典 (翻訳), 高木正弘 (翻訳), [Clean Architecture 達人に学ぶソフトウェアの構造と設計](https://www.kadokawa.co.jp/product/301806000678/), KADOKAWA, 2018.
*  ひらまつしょうたろう, [Python で身につける オブジェクト指向【SOLID原則+デザインパターンで、オブジェクト指向設計 の基礎を習得！】](https://www.udemy.com/course/python-solid-design-pattern/?couponCode=KEEPLEARNING), Udemy, 最終更新日 2024/6.
*  Mark Summerfield (著), 斎藤康毅 (訳), [実践 Python 3](https://www.oreilly.co.jp/books/9784873117393/), オライリージャパン, 2015.
<!-- *  Bill Lubanovic (著), 鈴木駿 (監訳), 長尾高弘 (訳), [入門 Python 3 第2版](https://www.oreilly.co.jp/books/9784873119328/), オライリージャパン, 2021. -->
<!-- *  Guido van Rossum (著), 鴨澤眞夫 (翻訳), [Pythonチュートリアル 第4版](https://www.oreilly.co.jp/books/9784873119359/), オライリージャパン, 2021. -->

